# Touch-Night/text-generation-webui

在运行所有单元格后，将在大约10分钟后在底部显示一个公共gradio URL。您还可以选择生成一个API链接。

* 原项目地址：https://github.com/oobabooga/text-generation-webui
* 汉化项目地址：https://github.com/Touch-Night/text-generation-webui/tree/Chinese
* Gradio服务器状态：https://status.gradio.app/

In [ ]:
#@title 1. 保持此标签页始终活动，防止Colab与你断开连接 { display-mode: "form" }

#@markdown 按下面的播放按钮：
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 2. 启动web UI

#@markdown 如果不确定branch，请填写“main”或留空。

import os
from pathlib import Path

os.environ.pop('PYTHONPATH', None)

if Path.cwd().name != 'text-generation-webui':
  print("\033[1;32;1m\n --> 正在安装web UI。这需要花点时间，但等初始环境配置好之后，你就可以随便把你喜欢的模型下载下来玩。\033[0;37;0m\n")

  !git clone -b ChineseHuggingface https://github.com/Touch-Night/text-generation-webui.git
  %cd text-generation-webui

  # 在隔离环境中安装此项目
  !GPU_CHOICE=A \
  USE_CUDA118=FALSE \
  LAUNCH_AFTER_INSTALL=FALSE \
  INSTALL_EXTENSIONS=FALSE \
  ./start_linux.sh

# 参数
model_url = "https://huggingface.co/turboderp/gemma-2-9b-it-exl2" #@param {type:"string"}
branch = "8.0bpw" #@param {type:"string"}
command_line_flags = "--n-gpu-layers 128 --load-in-4bit --use_double_quant --no_flash_attn" #@param {type:"string"}
api = False #@param {type:"boolean"}

if api:
  for param in ['--api', '--public-api']:
    if param not in command_line_flags:
      command_line_flags += f" {param}"

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    # 下载模型
    url_parts = model_url.strip('/').strip().split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = branch.strip('"\' ')
    if branch.strip() not in ['', 'main']:
        output_folder += f"_{branch}"
        !python download-model.py {model_url} --branch {branch}
    else:
        !python download-model.py {model_url}
else:
    output_folder = ""

# 启动web UI
cmd = f"./start_linux.sh {command_line_flags} --share"
if output_folder != "":
    cmd += f" --model {output_folder}"

!$cmd